In [1]:
# EICU
import pandas as pd
pd.set_option('display.max_colwidth', None)
dir_data_raw = "/Users/rachellehorwitz/Documents/VTAlert/data/raw_eicu/"
dir_data_over18 = "/Users/rachellehorwitz/Documents/VTAlert/data/over18_eicu/"
# admissiondx = pd.read_csv(dir_data_raw + 'admissionDx.csv')
# apachePredVar = pd.read_csv(dir_data_raw + 'apachePredVar.csv')
# diagnosis = pd.read_csv(dir_data_raw + 'diagnosis.csv')
# customLab = pd.read_csv(dir_data_raw + 'customLab.csv')

In [2]:
# Get patientUnitStayIds of patients who are >18 years old
# everything except hospital.csv can be indexed by patientUnitStayID
# patient.csv has age
import glob, os
import pandas as pd
csvfiles = []
for file in glob.glob(dir_data_raw + '*.csv'):
    csvfiles.append(file)
patient = pd.read_csv(dir_data_raw + 'patient.csv')
patient = patient[patient['age'].notna()]
print(str(patient.shape[0]) + ' patients with age not na')

# Patients with age > 89 do not have their actual age. They are just listed as '> 89'
# In MIMIC, it was known that the median age of patients > 89 was 91. Use that value here
over89 = patient[patient['age']=='> 89']
print(str(over89.shape[0]) + ' patients with age > 89')
patient['age'] = patient['age'].replace('> 89', '91').astype(int)
patientOver18 = patient[patient['age'] > 18]
print(str(patientOver18.shape[0]) + ' patients with age > 18')
patientUnitStayIdOver18 = list(patientOver18['patientunitstayid'])

200764 patients with age not na
7081 patients with age > 89
199702 patients with age > 18


In [ ]:
print(csvfiles)

In [ ]:
# import pandas as pd
# import os
# print(csvfiles[0])
# data = pd.read_csv(csvfiles[0])
# data.head(30)
# data2 = data[data['patientunitstayid'].isin(patientUnitStayIdOver18)]
# base = os.path.basename(csvfiles[0])
# print(base)
# newFileNameWithPath = dir_data_over18 + base
# print(newFileNameWithPath)
# data2.to_csv(newFileNameWithPath, index=False)

In [3]:
from rewrite_patientsOver18 import *
keepPtsOver18(csvfiles, patientUnitStayIdOver18, '/Users/rachellehorwitz/Documents/tmp/', dir_data_raw)

------
Working on /Users/rachellehorwitz/Documents/VTAlert/data/raw_eicu/medication.csv


/Users/rachellehorwitz/opt/anaconda3/envs/insight/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


medication.csv


KeyboardInterrupt: 

In [ ]:
print(csvfiles)

In [ ]:
admissiondx = pd.read_csv(dir_data_over18 + 'admissiondx.csv')
diagnosis = pd.read_csv(dir_data_over18 + 'diagnosis.csv')

In [ ]:
# import glob, os
# os.chdir(dir_data_raw)
# for file in glob.glob('*.csv'):
#     print(file)

In [ ]:
admissiondx.head(30)

In [ ]:
print('there are ' + str(diagnosis.shape[0]) +  ' diagnosis entries')
diagnosis.head(10)

In [ ]:
diagnoses_nonan = diagnosis[diagnosis['icd9code'].notna()]
print('there are ' + str(diagnoses_nonan.shape[0]) +  ' diagnosis entries with non-nan icd9codes')
diagnoses_nonan.head(5)

In [ ]:
icd9_pe = [415.1, 415.10, 415.11, 415.13, 415.19, 416.2, 416.20]
icd9_dvt = [451.0, 451.1, 451.10, 451.11, 451.19, 451.2, 451.20, 451.81, 451.89, \
            451.9, 451.90, 453.1, 453.10, 453.2, 453.20, 453.4, 453.40, 453.41, \
           453.42, 453.6, 453.60, 453.8, 453.80, 453.89, 453.9, 453.90] # excludes chronic
icd9_vte = icd9_pe + icd9_dvt
icd9_vte_str = [str(x) for x in icd9_vte] 

In [ ]:
icd9_pe = ['415.1', '415.10', '415.11', '415.13', '415.19']
icd9_dvt = ['451.0', '451.1', '451.10', '451.11', '451.19', '451.2', '451.20', '451.81', '451.89', \
            '451.9', '451.90', '453.1', '453.10', '453.2', '453.20', '453.4', '453.40', '453.41', \
           '453.42', '453.6', '453.60', '453.8', '453.80', '453.89', '453.9', '453.90'] # excludes chronic
icd9_acute_dvt = ['453.4', '453.89']
icd9_vte = icd9_pe + icd9_acute_dvt

In [ ]:
diagnosis_vte = pd.DataFrame()
for code in icd9_vte:
    print(str(code))
    rows = diagnoses_nonan[diagnoses_nonan['icd9code'].str.contains(code)]
    diagnosis_vte = pd.concat([rows, diagnosis_vte])
    diagnosis_vte = diagnosis_vte.loc[~diagnosis_vte.index.duplicated(keep='first')]

In [ ]:
diagnosis_vte.shape

In [ ]:
diagnosis_vte.head(30)

In [ ]:
%matplotlib notebook
diagnosis_vte['diagnosisoffset'].hist(bins=500)

In [ ]:
import numpy as np
# Don't look at patients who are entering the ICU for PE/DVT, so threshold by time
time_thresh = 400
vte_greaterThanTime = diagnosis_vte[diagnosis_vte['diagnosisoffset']>=time_thresh]
vte_lessThanTime = diagnosis_vte[diagnosis_vte['diagnosisoffset']<time_thresh]
print(str(greaterThanTime.shape[0]) + ' visits are included for time_thresh=' + str(time_thresh))
print(str(lessThanTime.shape[0]) + ' visits are eliminated for time_thresh=' + str(time_thresh))

# Look for duplicates in patientunitstayid in vte_greaterThanTime and vte_lessThanTime to see if some people were diagnosed more than once
u_vte_greaterThanTime_patientunitstayid = vte_greaterThanTime['patientunitstayid'].unique()
print(str(u_vte_greaterThanTime_patientunitstayid.shape[0]) + ' unique patientunitstayids in greaterThanTime')
u_vte_lessThanTime_patientunitstayid = vte_lessThanTime['patientunitstayid'].unique()
print(str(u_vte_lessThanTime_patientunitstayid.shape[0]) + ' unique patientunitstayids in lessThanTime')
type(u_vte_lessThanTime_patientunitstayid)
patientunitstayid_intersect = np.intersect1d(u_vte_greaterThanTime_patientunitstayid, u_vte_lessThanTime_patientunitstayid)
print(str(patientunitstayid_intersect.shape[0]) + ' patientunitstayids in both')

In [ ]:
# Intermediate goal: get two classes:
# 1) patientunitstayids that have VTE diagnosed after a while in the ICU: *u_vte_greaterThanTime*
# 2) patientunitstayids that do not have VTE diagnosed after a while in the ICU. 
#    all_patientunitstayids - union(less, greater)
